In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
import random
import shap
import sys
import seaborn as sns

In [ ]:
from sklearn.model_selection import StratifiedKFold
from scipy.stats import pearsonr, spearmanr, shapiro, ttest_ind, mannwhitneyu
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.inspection import permutation_importance
from matplotlib.patches import Patch

from sklearn.feature_selection import RFECV
from deap import creator, base, tools, algorithms

from statistics import mean, stdev
from math import sqrt
from statsmodels.stats.multitest import multipletests
from cliffs_delta import cliffs_delta

In [ ]:
random.seed(42)
np.random.seed(42)

In [ ]:
%matplotlib inline

In [ ]:
# path = ...
df = pd.read_csv(path,sep=',')
df.drop(['CC'],axis = 1, inplace = True)

In [ ]:

cv = StratifiedKFold(n_splits=3)

In [ ]:
data = df.drop(['examinee'], axis=1)
cols = data.columns
n = len(cols)
pearson_corr_mat = pd.DataFrame(np.eye(n), index=cols, columns=cols)
pearson_pval_mat = pd.DataFrame(np.zeros((n, n)), index=cols, columns=cols)

for i, xi in enumerate(cols):
    for j, xj in enumerate(cols):
        if i <= j:
            r, p = pearsonr(data[xi], data[xj])
            pearson_corr_mat.iat[i, j] = pearson_corr_mat.iat[j, i] = r
            pearson_pval_mat.iat[i, j] = pearson_pval_mat.iat[j, i] = p

annot = pearson_pval_mat.applymap(lambda p: '*' if p < 0.05 else '')

mask = np.triu(np.ones_like(pearson_corr_mat, dtype=bool))

plt.figure(figsize=(10, 8))
ax = sns.heatmap(
    pearson_corr_mat,
    mask=mask,
    annot=annot,
    fmt='',
    cmap='cividis',
    vmin=-1, vmax=1,
    square=True,
    cbar_kws={"shrink": 0.8}
)

ax.set_xticklabels(cols, rotation=90)
ax.set_yticklabels(cols, rotation=0)

plt.title(
"""Pearson's r coefficient""",
    fontsize=18
)
plt.tight_layout()
plt.show()

In [ ]:
mask = np.tril(np.ones(pearson_corr_mat.shape), k=-1).astype(bool)

high_corr_pairs = (np.abs(pearson_corr_mat.values)[mask] > 0.9).sum()

print("Number of pairs with |corr| > 0.9:", high_corr_pairs)

In [ ]:
data = df.drop(['examinee'], axis=1)
cols = data.columns
n = len(cols)

spearman_rho_mat  = pd.DataFrame(np.eye(n), index=cols, columns=cols)
spearman_pval_mat = pd.DataFrame(np.zeros((n, n)), index=cols, columns=cols)

for i, xi in enumerate(cols):
    for j, xj in enumerate(cols):
        if i <= j:
            rho, p = spearmanr(data[xi], data[xj])
            spearman_rho_mat.iat[i, j]  = spearman_rho_mat.iat[j, i]  = rho
            spearman_pval_mat.iat[i, j] = spearman_pval_mat.iat[j, i] = p

annot = spearman_pval_mat.applymap(lambda p: '*' if p < 0.05 else '')

mask = np.triu(np.ones_like(spearman_rho_mat, dtype=bool))

plt.figure(figsize=(10, 8))
ax = sns.heatmap(
    spearman_rho_mat,
    mask=mask,
    annot=annot,
    fmt='',
    cmap='cividis',
    vmin=-1, vmax=1,
    square=True,
    cbar_kws={"shrink": 0.8}
)

ax.set_xticklabels(cols, rotation=90)
ax.set_yticklabels(cols, rotation=0)

plt.title(
"Spearman's $\\rho$ coefficient\n",
    fontsize=18
)
plt.tight_layout()
plt.show()

In [ ]:
mask = np.tril(np.ones(spearman_rho_mat.shape), k=-1).astype(bool)

high_corr_pairs = (np.abs(spearman_rho_mat.values)[mask] > 0.9).sum()

print("Number of pairs with |corr| > 0.9:", high_corr_pairs)

In [ ]:
features = pearson_corr_mat.columns

# Set diagonals
for mat in [pearson_corr_mat, spearman_rho_mat]:
    np.fill_diagonal(mat.values, 1.0)
for mat in [pearson_pval_mat, spearman_pval_mat]:
    np.fill_diagonal(mat.values, 0.0)

threshold = 0.7
pairs = []

for i in range(len(features)):
    for j in range(i+1, len(features)):
        p_corr = pearson_corr_mat.iloc[i, j]
        s_corr = spearman_rho_mat.iloc[i, j]
        p_pval = pearson_pval_mat.iloc[i, j]
        s_pval = spearman_pval_mat.iloc[i, j]

        p_flag = abs(p_corr) > threshold
        s_flag = abs(s_corr) > threshold

        if p_flag or s_flag:
            if p_flag and s_flag:
                method = "Both"
            elif p_flag:
                method = "Pearson only"
            else:
                method = "Spearman only"

            pairs.append({
                "Feature 1": features[i],
                "Feature 2": features[j],
                "Pearson": round(p_corr, 3),
                "Spearman": round(s_corr, 3),
                "Pearson pval": round(p_pval, 3),
                "Spearman pval": round(s_pval, 3),
                "Method": method
            })

pairs_df = pd.DataFrame(pairs)

pairs_df = pairs_df.sort_values(by="Pearson", ascending=False).reset_index(drop=True)

print(pairs_df)

In [ ]:
df_baseline = df[df['examinee'].str.endswith('- 1')].copy()
df_anger = df[df['examinee'].str.endswith('- 2')].copy()

df_baseline['examinee'] = df_baseline['examinee'].str.replace(' - 1', '', regex=False)
df_anger['examinee'] = df_anger['examinee'].str.replace(' - 2', '', regex=False)

In [ ]:
df['examinee'] = df['examinee'].str.replace(' - 1', '', regex=False)
df['examinee'] = df['examinee'].str.replace(' - 2', '', regex=False)

In [ ]:
X = df.drop('examinee',axis=1)
# y = df['examinee']

In [ ]:
df['examinee'].value_counts()

In [ ]:
with open('subject_ID.pkl', 'rb') as f:
    le = pickle.load(f)
    
y = le.transform(df['examinee'])

In [ ]:
rf = RandomForestClassifier(n_jobs=-1, random_state=42)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, stratify=y, random_state=42)

rf.fit(X_train, y_train)

y_pred = rf.predict(X_test)

accuracy = accuracy_score(y_test, y_pred) * 100
f1 = f1_score(y_test, y_pred, average='weighted') * 100
precision = precision_score(y_test, y_pred, average='weighted') * 100
recall = recall_score(y_test, y_pred, average='weighted') * 100

print(f"Accuracy: {accuracy:.2f}")
print(f"F1 Score: {f1:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")

In [ ]:
feature_importances = rf.feature_importances_

feature_names = X_train.columns
gini_importance_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': feature_importances
}).sort_values(by='Importance', ascending=False)

rf_top_10 = set(gini_importance_df.sort_values(by='Importance',ascending=False).reset_index(drop=True)['Feature'].iloc[:10])

In [ ]:
rf = RandomForestClassifier(n_jobs=-1, random_state=42)

rf.fit(X_train, y_train)

perm_importance = permutation_importance(rf, X_test, y_test, n_repeats=100, random_state=42, n_jobs=-1)

perm_importance_df = pd.DataFrame({
    'Feature': X_test.columns,
    'Importance': perm_importance.importances_mean,
    'StdDev': perm_importance.importances_std
}).sort_values(by='Importance', ascending=False)

perm_importance_df = perm_importance_df.sort_values(by='Importance', ascending=False, ignore_index=True)
perm_top_10 = set(perm_importance_df['Feature'].iloc[:10])

In [ ]:
perm_top_10

In [ ]:
rf = RandomForestClassifier(n_jobs=-1, random_state=42)
rf.fit(X_train, y_train)

explainer = shap.Explainer(rf)
shap_values = explainer(X_test)

vals = shap_values.values

if vals.ndim == 3:
    vals = vals.mean(axis=2)

importance = np.abs(vals).mean(axis=0)

shap_importance_df = pd.DataFrame({
    'Feature': X_test.columns,
    'Importance': importance
}).sort_values(by='Importance', ascending=False, ignore_index=True)

shap_top_10 = shap_importance_df['Feature'].iloc[:10].tolist()
print(shap_top_10)

In [ ]:
temp = shap_values.values


In [ ]:
plt.figure(figsize=(12, 8))

sorted_features = shap_importance_df['Feature']
sorted_importance = shap_importance_df['Importance']

colors = ['#08306b' if feat in shap_top_10 else 'gray' for feat in sorted_features]

bars = plt.barh(sorted_features, sorted_importance, color=colors)
plt.gca().invert_yaxis()

plt.xlabel("Mean |SHAP value|", fontsize=14)
plt.ylabel("Features", fontsize=14)
plt.title("SHAP feature importance", fontsize=21)

plt.yticks(fontsize=14)
plt.xticks(fontsize=12)

legend_elements = [
    Patch(facecolor='#08306b', label='Top 10 Features'),
    Patch(facecolor='gray', label='Remaining Features')
]
plt.legend(handles=legend_elements, fontsize=12, loc="lower right")

plt.tight_layout()
plt.show()

In [ ]:
all_three_intersection = rf_top_10.intersection(shap_top_10).intersection(perm_top_10)
print(all_three_intersection)

In [ ]:
# Example sets
set1 = set(rf_top_10)
set2 = set(perm_top_10)
set3 = set(shap_top_10)

inter_all = set1 & set2 & set3
print("Intersection of all three:", inter_all)

# Intersection of each two (excluding all-three)
inter_12 = (set1 & set2) - inter_all
inter_13 = (set1 & set3) - inter_all
inter_23 = (set2 & set3) - inter_all
print("Intersection set1 & set2 only:", inter_12)
print("Intersection set1 & set3 only:", inter_13)
print("Intersection set2 & set3 only:", inter_23)

# Unique elements of each set
unique1 = set1 - (set2 | set3)
unique2 = set2 - (set1 | set3)
unique3 = set3 - (set1 | set2)
print("Unique in set1:", unique1)
print("Unique in set2:", unique2)
print("Unique in set3:", unique3)

left_by_all = set(X.columns) - (set1 | set2 | set3)
print("Left out by all three:", left_by_all)

In [ ]:
X_three_intersection = X[list(all_three_intersection)]

rf = RandomForestClassifier(n_jobs=-1, random_state=42)

X_train, X_test, y_train, y_test = train_test_split(X_three_intersection, y, test_size=0.33, stratify=y, random_state=42)

rf.fit(X_train, y_train)

y_pred = rf.predict(X_test)

accuracy = accuracy_score(y_test, y_pred) * 100
f1 = f1_score(y_test, y_pred, average='weighted') * 100
precision = precision_score(y_test, y_pred, average='weighted') * 100
recall = recall_score(y_test, y_pred, average='weighted') * 100

print(f"Accuracy: {accuracy:.2f}")
print(f"F1 Score: {f1:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, stratify=y, random_state=42)

In [ ]:
colors = []
for feature in gini_importance_df['Feature']:
    if feature in rf_top_10:
        colors.append('#00224E')

    else:
        colors.append('gray') 

plt.figure(figsize=(12, 8))
bars = plt.barh(
    gini_importance_df['Feature'], 
    gini_importance_df['Importance'], 
    align='center', 
    color=colors,
    capsize=5
)

plt.gca().invert_yaxis()
plt.xlabel('Importance score', fontsize = 14)
plt.ylabel('Feature name', fontsize = 14)
plt.title('Gini importance', fontsize = 21)

for label in plt.gca().get_yticklabels():
    label.set_fontsize(14)
    
legend_elements = [
    Patch(facecolor='#00224E', label='Top 10 features'),
    Patch(facecolor='gray', label='Remaining features')
]
plt.legend(handles=legend_elements, loc='lower right', fontsize = 12)

plt.tight_layout()
plt.show()

In [ ]:
colors = []
for feature in perm_importance_df['Feature']:
    if feature in perm_top_10:
        colors.append('#00224E')
    else:
        colors.append('gray')

plt.figure(figsize=(12, 8))
bars = plt.barh(
    perm_importance_df['Feature'], 
    perm_importance_df['Importance'], 
    xerr=perm_importance_df['StdDev'], 
    align='center', 
    color=colors,
    capsize=5
)

for label in plt.gca().get_yticklabels():
    label.set_fontsize(14)
    
plt.gca().invert_yaxis()
plt.xlabel('Mean reduction in accuracy', fontsize = 14)
plt.ylabel('Feature name', fontsize = 14)
plt.title('Permutation importance', fontsize = 21)

legend_elements = [
    Patch(facecolor='#00224E', label='Top 10 features'),
    Patch(facecolor='gray', label='Remaining features')
]
plt.legend(handles=legend_elements, loc='lower right', fontsize = 12)

plt.tight_layout()
plt.show()


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, stratify=y, random_state=42)

rf = RandomForestClassifier(random_state=42, n_jobs=-1)
rfecv = RFECV(estimator=rf, step=1, cv=cv, scoring='accuracy', n_jobs=-1)

rfecv.fit(X_train, y_train)

rfe_feature_set = X_train.columns[rfecv.support_]
print("Optimal number of features:", rfecv.n_features_)
print("Selected features:", list(rfe_feature_set))

In [ ]:
set(X_train.columns.to_list()) - set(rfe_feature_set)

In [ ]:
rf_selected = RandomForestClassifier(random_state=42, n_jobs=-1)
rf_selected.fit(X_train[rfe_feature_set], y_train)

y_pred = rf_selected.predict(X_test[rfe_feature_set])

accuracy = accuracy_score(y_test, y_pred) * 100
f1 = f1_score(y_test, y_pred, average='weighted') * 100
precision = precision_score(y_test, y_pred, average='weighted') * 100
recall = recall_score(y_test, y_pred, average='weighted') * 100

print(f"Accuracy: {accuracy:.2f}")
print(f"F1 Score: {f1:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")

In [ ]:
# The feature-selection process below uses a Genetic Algorithm (GA) implementation
# adapted for this project. The GA functions used here are based on the publicly
# available implementation from:
#     https://github.com/Luck032/GeneticAlgorithmForFeatureSelection
#
# The structure of the algorithm, including the evaluation strategy and 
# representation of individuals, is also largely inspired by:
#     https://github.com/renatoosousa/GeneticAlgorithmForFeatureSelection
#
# Both repositories provide general-purpose frameworks for GA-based feature selection.
# The code here follows their logic but has been adapted to fit the requirements of
# this application (e.g., defined CV strategy, model choice, and data handling).
#
# Users are free to adopt, modify, or extend these components as appropriate for
# their own machine learning workflows.

random.seed(42)
np.random.seed(42)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, stratify=y, random_state=42
)

individual = [1 for i in range(len(X_train.columns))]
print("Accuracy with all features: \t" +
      str(getFitness(individual, X_train, y_train)) + "\n")

n_gen = 50
n_pop = 50

hof = geneticAlgorithm(X_train, y_train, n_pop, n_gen)

accuracy, individual, header = bestIndividual(hof, X_train, y_train)
print('Number of Features in Subset: \t' + str(individual.count(1)))
print('Feature Subset\t: ' + str(header))


In [ ]:
ga_feature_set = set(['QRS_int', 'T_int', 'QT_int', 'RS_amp', 'RQ_amp', 'RT_amp', 'TT1_amp', 'TT2_amp', 'QR_slope', 'RS_slope', 'ECGQRScrest', 'ECGTcrest', 'CX_amp', 'CB_slope', 'BT_int', 'TX_int', 'RR'])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, stratify=y, random_state=42)

X_train_selected = X_train[list(ga_feature_set)]
X_test_selected = X_test[list(ga_feature_set)]

rf = RandomForestClassifier(random_state=42, n_jobs=-1)
rf.fit(X_train_selected, y_train)

y_pred = rf.predict(X_test_selected)

accuracy = accuracy_score(y_test, y_pred) *100
precision = precision_score(y_test, y_pred, average='macro') *100
recall = recall_score(y_test, y_pred, average='macro') *100
f1 = f1_score(y_test, y_pred, average='macro') *100

print(f"Accuracy: {accuracy:.2f}")
print(f"F1 Score: {f1:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")

In [ ]:
ga_feature_set = set(ga_feature_set)
rfe_feature_set = set(rfe_feature_set)

print(len(set.intersection(rfe_feature_set, ga_feature_set)))
print(set.intersection(rfe_feature_set, ga_feature_set))

In [ ]:
ga_feature_set - set.intersection(rfe_feature_set, ga_feature_set)

In [ ]:
rfe_feature_set - set.intersection(rfe_feature_set, ga_feature_set)

In [ ]:
set(X.columns) - rfe_feature_set.union(ga_feature_set)

In [ ]:
print(len(set.intersection(rfe_feature_set, ga_feature_set).intersection(all_three_intersection)))
print(set.intersection(rfe_feature_set, ga_feature_set).intersection(all_three_intersection))

In [ ]:
all_features = set(df.drop('examinee',axis=1).columns.to_list())

In [ ]:
selected = list(set.intersection(rfe_feature_set, ga_feature_set))

rf = RandomForestClassifier(random_state=42, n_jobs=-1)
rf.fit(X_train[selected], y_train)

y_pred = rf.predict(X_test[selected])

accuracy = accuracy_score(y_test, y_pred) *100
precision = precision_score(y_test, y_pred, average='macro') *100
recall = recall_score(y_test, y_pred, average='macro') *100
f1 = f1_score(y_test, y_pred, average='macro') *100

print(f"Accuracy: {accuracy:.2f}")
print(f"F1 Score: {f1:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")

# Contribtions of clusters of highly correlated features to biometric identification

In [ ]:
correlation_matrix_all_subject = df.drop('examinee',axis=1).corr(method='pearson')

upper_triangle = correlation_matrix_all_subject.where(np.triu(np.ones(correlation_matrix_all_subject.shape), k=1).astype(bool))

threshold = 0.7
high_corr_pairs = upper_triangle.stack().reset_index()
high_corr_pairs.columns = ['Variable 1', 'Variable 2', 'Correlation']
high_corr_pairs = high_corr_pairs[high_corr_pairs['Correlation'].abs() > threshold]

high_corr_pairs = high_corr_pairs.sort_values(by = 'Correlation', ascending=False).reset_index(drop=True)

high_corr_pairs['Correlation'] = high_corr_pairs['Correlation'].round(2)

print(high_corr_pairs.sort_values(by = 'Correlation', ascending=False))

In [ ]:
pd.concat([high_corr_pairs['Variable 1'], high_corr_pairs['Variable 2']]).value_counts()

In [ ]:
all_variables = X.columns

adjacency = {}
for _, row in high_corr_pairs.iterrows():
    if row["Variable 1"] not in adjacency:
        adjacency[row["Variable 1"]] = []
    if row["Variable 2"] not in adjacency:
        adjacency[row["Variable 2"]] = []
    adjacency[row["Variable 1"]].append(row["Variable 2"])
    adjacency[row["Variable 2"]].append(row["Variable 1"])

def dfs(node, cluster_id, visited, clusters):
    visited.add(node)
    clusters[node] = cluster_id
    for neighbor in adjacency.get(node, []):
        if neighbor not in visited:
            dfs(neighbor, cluster_id, visited, clusters)

visited = set()
clusters = {}
cluster_id = 0
for variable in adjacency.keys():
    if variable not in visited:
        cluster_id += 1
        dfs(variable, cluster_id, visited, clusters)

for var in all_variables:
    if var not in clusters:
        cluster_id += 1
        clusters[var] = cluster_id

cluster_result_df = pd.DataFrame(list(clusters.items()), columns=["Var", "Cluster"])
cluster_result_df.sort_values(by="Cluster", inplace=True)

In [ ]:
cluster_result_df['Cluster'].value_counts()

In [ ]:
cluster_result_df[cluster_result_df['Cluster'] == 6]['Var']

In [ ]:
extract_feature = 'SB_int'
feature_pairs = high_corr_pairs[
    (high_corr_pairs['Variable 1'] == extract_feature) |
    (high_corr_pairs['Variable 2'] == extract_feature)
]

high_corr_list = pd.concat([feature_pairs['Variable 1'], feature_pairs['Variable 2']])

high_corr_list = high_corr_list[high_corr_list != extract_feature].reset_index(drop=True)

print(feature_pairs.head())
print(high_corr_list.head())

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, stratify=y, random_state=42)

rf = RandomForestClassifier(random_state=42, n_jobs=-1)
rf.fit(X_train, y_train)
original_importances = pd.Series(rf.feature_importances_, index=X.columns)


X_train_permuted = X_train.copy()
feature_perm = extract_feature
X_train_permuted[feature_perm] = np.random.permutation(X_train_permuted[feature_perm])

rf_permuted = RandomForestClassifier(random_state=42, n_jobs=-1)
rf_permuted.fit(X_train_permuted, y_train)
permuted_importances = pd.Series(rf_permuted.feature_importances_, index=X.columns)


importance_comparison = pd.DataFrame({
    "Original Importance": original_importances,
    "Permuted Importance": permuted_importances,
    "Difference": 100 * (permuted_importances - original_importances) / original_importances
})

In [ ]:
perm_cluster = pd.concat((high_corr_list,pd.Series(extract_feature))).reset_index(drop=True)
print('Shuffeld features:',perm_cluster.to_list())

original_test_accuracy = accuracy_score(y_test, rf.predict(X_test))

X_test_permuted = X_test.copy()
rng = np.random.default_rng(42)                  
perm_idx = rng.permutation(len(X_test_permuted))           
X_test_permuted[perm_cluster] = X_test_permuted[perm_cluster].to_numpy()[perm_idx, :]

permuted_test_accuracy = accuracy_score(y_test, rf.predict(X_test_permuted))

difference_in_accuracy = (permuted_test_accuracy - original_test_accuracy) / original_test_accuracy * 100

print("Original Test Set Accuracy:", original_test_accuracy * 100)
print("Permuted Test Set Accuracy:", permuted_test_accuracy * 100)
print("Difference in Accuracy:", difference_in_accuracy)

In [ ]:
print(high_corr_list)

In [ ]:
importance_comparison.loc[high_corr_list, 'Difference'].round(2)

In [ ]:
high_corr_list = pd.concat((high_corr_list,pd.Series(extract_feature))).reset_index(drop=True)
high_corr_list

In [ ]:
boolean_mask = ~importance_comparison.index.isin(high_corr_list)

importance_comparison.loc[boolean_mask, 'Difference'].describe().round(2)

# Effect of emotions on cardiogram-based features 

In [ ]:
def cohen_d(c0, c1):
    d = (mean(c0) - mean(c1)) / (sqrt((stdev(c0) ** 2 + stdev(c1) ** 2) / 2))

    abs_d = abs(d)
    if abs_d < 0.20:
        interpretation = "Negligible effect"
    elif abs_d < 0.50:
        interpretation = "Small effect"
    elif abs_d < 0.80:
        interpretation = "Medium effect"
    else:
        interpretation = "Large effect"
    
    return d, interpretation

In [ ]:
features = df_baseline.drop('examinee', axis=1).columns
results = []

alpha = 0.05

for feature in features:
    data_baseline = df_baseline[feature].dropna()
    data_anger = df_anger[feature].dropna()

    _, p_base = shapiro(data_baseline)
    _, p_anger = shapiro(data_anger)

    normal_base = p_base > alpha
    normal_anger = p_anger > alpha

    if normal_base and normal_anger:
        # t-test + Cohen's d
        stat, p_val = ttest_ind(data_baseline, data_anger, equal_var=False)
        d, interp = cohen_d(data_baseline, data_anger)
        eff, eff_type, eff_interp = d, "Cohen's d", interp
        test_used = "t-test (Welch)"
    else:
        # Mann-Whitney + Cliff's delta
        stat, p_val = mannwhitneyu(data_baseline, data_anger, alternative='two-sided')
        delta, size = cliffs_delta(data_baseline, data_anger)
        eff, eff_type, eff_interp = delta, "Cliff's delta", size
        test_used = "Mann-Whitney U"

    results.append({
        'Feature': feature,
        'Test': test_used,
        'p_value': p_val.round(3),
        'Effect': eff,
        'Effect Type': eff_type,
        'Effect Interpretation': eff_interp
    })

# DataFrame
results_df = pd.DataFrame(results)

# Bonferroni correction
reject, p_adj, _, _ = multipletests(results_df['p_value'], method='bonferroni', alpha=alpha)
results_df['p_adj'] = p_adj.round(3)
results_df['Significant'] = reject

# Split views
significant_features = results_df[results_df['Significant'] == True]
nonsignificant_features = results_df[results_df['Significant'] == False]

print("\nSignificant features:\n", significant_features)
print("\nNonsignificant features:\n", nonsignificant_features)

In [ ]:
nonsignificant_features['Feature'].to_list()

In [ ]:
nonsignificant_features['Feature']

In [ ]:
opposite_significance_pairs = []

for _, row in high_corr_pairs.iterrows():
    var1 = row['Variable 1']
    var2 = row['Variable 2']
    
    var1_row = results_df.loc[results_df['Feature'] == var1].iloc[0]
    significant_var1 = var1_row['Significant']
    adj_p_value_var1 = var1_row['p_adj']
    
    var2_row = results_df.loc[results_df['Feature'] == var2].iloc[0]
    significant_var2 = var2_row['Significant']
    adj_p_value_var2 = var2_row['p_adj']
    
    if significant_var1 != significant_var2:
        opposite_significance_pairs.append({
            'Variable 1': var1,
            'Variable 2': var2,
            'Correlation': row['Correlation'],
            'Variable 1 Significant': significant_var1,
            'Variable 2 Significant': significant_var2,
            'Variable 1 Adjusted p_value': adj_p_value_var1,
            'Variable 2 Adjusted p_value': adj_p_value_var2
        })

opposite_significance_df = pd.DataFrame(opposite_significance_pairs)

In [ ]:
del X_train, X_test

In [ ]:
# featureset = nonsignificant_features['Feature']
# featureset = df.drop('examinee', axis=1).columns
featureset = significant_features['Feature']

with open('subject_ID.pkl', 'rb') as f:
    le = pickle.load(f)

X_train = df_baseline[featureset]
y_train = le.transform(df_baseline['examinee'])

X_test = df_anger[featureset]
y_test = le.transform(df_anger['examinee'])

rf = RandomForestClassifier(n_jobs=-1, random_state=42)

acc_scores = cross_val_score(rf, X_train, y_train, cv=cv, scoring='accuracy')
f1_scores = cross_val_score(rf, X_train, y_train, cv=cv, scoring='f1_weighted')

print(f"CV Accuracy: {acc_scores.mean()*100:.2f} ± {acc_scores.std()*100:.2f}")
print(f"CV F1 Score: {f1_scores.mean()*100:.2f} ± {f1_scores.std()*100:.2f}")

rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)

test_acc = accuracy_score(y_test, y_pred) * 100
test_f1 = f1_score(y_test, y_pred, average='weighted') * 100

print(f"Test Accuracy (unseen): {test_acc:.2f}")
print(f"Test F1 Score (unseen): {test_f1:.2f}")


In [ ]:
del X_train, X_test

In [ ]:
with open('subject_ID.pkl', 'rb') as f:
    le = pickle.load(f)

# Split train/test
X_test = df_baseline[featureset]
y_test = le.transform(df_baseline['examinee'])

X_train = df_anger[featureset]
y_test = le.transform(df_anger['examinee'])

# Model
rf = RandomForestClassifier(n_jobs=-1, random_state=42)

acc_scores = cross_val_score(rf, X_train, y_train, cv=cv, scoring='accuracy')
f1_scores = cross_val_score(rf, X_train, y_train, cv=cv, scoring='f1_weighted')

print(f"CV Accuracy: {acc_scores.mean()*100:.2f} ± {acc_scores.std()*100:.2f}")
print(f"CV F1 Score: {f1_scores.mean()*100:.2f} ± {f1_scores.std()*100:.2f}")

rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)

test_acc = accuracy_score(y_test, y_pred) * 100
test_f1 = f1_score(y_test, y_pred, average='weighted') * 100

print(f"Test Accuracy (unseen): {test_acc:.2f}")
print(f"Test F1 Score (unseen): {test_f1:.2f}")
